In [1]:
import numpy as np

In [ ]:
def compute_loss(y, tx, w):
    err=y-tx.dot(w)
    N=len(y)
    return 0.5*np.power(err,2).sum()/N

def batch_iter(y, tx, batch_size=1, num_batches=1, shuffle=True):
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

def compute_gradient(y, tx, w):
    err = y-tx.dot(w)
    N = len(y)
    return -tx.T.dot(err)/N

### end of helpers ###

def mean_squared_error_gd(y, tx, initial_w, max_iters, gamma):
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and loss
        gradient = compute_gradient(y,tx,w)
        loss = compute_loss(y,tx,w)
        # update w
        w = w-gamma*gradient

        # store w and loss
        ws.append(w)
        losses.append(loss)

    return losses, ws

def mean_squared_error_sgd(y, tx, initial_w, max_iters, gamma):
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w

    for n_iter in range(max_iters):
        for yb, xb in batch_iter(y,tx):
            # compute gradient on batch and loss
            stochastic_grad=compute_gradient(yb,xb,w)
            loss=compute_loss(yb,xb,w)
            # update w
            w=w-gamma*stochastic_grad

            ws.append(w)
            losses.append(loss)

    return losses, ws

def least_squares(y, tx):
    w_star = np.linalg.solve(tx.T.dot(tx),tx.T.dot(y))
    loss = compute_loss(y,tx,w_star)
    return w_star, loss

def ridge_regression(y, tx, _lambda):
    N = int(tx.size/tx.shape[0])
    lambda__ = lambda_*2*N
    return np.linalg.solve(tx.T.dot(tx)+lambda__*np.eye(N), tx.T.dot(y))